In [1]:
#!pip install cbgen

In [2]:
# https://github.com/limix/cbgen-py/blob/master/cbgen/test/test_bgen.py
from cbgen import bgen_file, bgen_metafile, example

In [3]:
path = '/home/eczech/data/rs-ukb-local/bgen/ukb_imp_chrXY_v3.bgen'

In [4]:
mf_path = '/tmp/ukb_imp_chrXY_v3.metafile'

In [6]:
bgen = bgen_file(path)

In [8]:
bgen.create_metafile(mf_path, verbose=True)

In [9]:
!du -ch $mf_path

2.5M	/tmp/ukb_imp_chrXY_v3.metafile
2.5M	total


In [10]:
mf = bgen_metafile(mf_path)

In [13]:
mf.partition_size

215

In [14]:
mf.npartitions

214

In [15]:
mf.nvariants

45906

In [40]:
bgen.nsamples

486443

In [41]:
bgen.nvariants

45906

In [42]:
mf.npartitions

214

In [33]:
#part = mf.read_partition(5)

part = mf.read_partition(213)

In [36]:
part.variants.id.shape

(111,)

In [24]:
part.variants.id[:10]

array([b'X:187693_G_A', b'X:187718_G_A', b'X:187740_G_C', b'X:187742_G_A',
       b'X:187751_G_C', b'X:187764_G_A', b'X:187764_G_T', b'X:187808_G_A',
       b'X:187812_C_T', b'X:187813_G_A'], dtype='|S12')

In [26]:
part.variants.chrom[:10]

array([b'XY', b'XY', b'XY', b'XY', b'XY', b'XY', b'XY', b'XY', b'XY',
       b'XY'], dtype='|S2')

In [46]:
part.variants.offset[:10]

array([4899590971, 4899694391, 4899728368, 4899738222, 4899879691,
       4900023440, 4900164915, 4900178515, 4900211847, 4900236628],
      dtype=uint64)

In [37]:
part.variants.offset[0]

4899590971

In [38]:
g = bgen.read_genotype(4899590971)
g

Genotype(probs=array([[1.        , 0.        , 0.        ],
       [0.98823529, 0.01176471, 0.        ],
       [1.        , 0.        , 0.        ],
       ...,
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ]]), phased=False, ploidy=array([2, 2, 2, ..., 2, 2, 2], dtype=uint8), missing=array([False, False, False, ..., False, False, False]))

In [39]:
g.probs.shape

(486443, 3)

In [48]:
??bgen.read_genotype

Signature: bgen.read_genotype(offset: 'int') -> 'Genotype'
Docstring: <no docstring>
Source:   
    def read_genotype(self, offset: int) -> Genotype:
        gt: CData = lib.bgen_file_open_genotype(self._bgen_file, offset)
        if gt == ffi.NULL:
            raise RuntimeError(f"Could not open genotype (offset {offset}).")

        nsamples = self.nsamples
        ncombs = lib.bgen_genotype_ncombs(gt)
        probs = full((nsamples, ncombs), nan, dtype=float64)
        err: int = lib.bgen_genotype_read(gt, ffi.cast("double *", probs.ctypes.data))
        if err != 0:
            msg = f"Could not read genotype probabilities (offset {offset})."
            raise RuntimeError(msg)

        phased = lib.bgen_genotype_phased(gt)

        ploidy = full(nsamples, 0, dtype=uint8)
        lib.read_ploidy(gt, ffi.cast("uint8_t *", ploidy.ctypes.data), nsamples)

        missing = full(nsamples, 0, dtype=bool)
        lib.read_missing(gt, ffi.cast("bool *", missing.ctypes.data), nsamples)

  

In [49]:
??mf.read_partition

Signature: mf.read_partition(index: 'int') -> 'Partition'
Docstring: <no docstring>
Source:   
    def read_partition(self, index: int) -> Partition:
        partition = lib.bgen_metafile_read_partition(self._bgen_metafile, index)
        if partition == ffi.NULL:
            raise RuntimeError(f"Could not read partition {partition}.")

        nvariants = lib.bgen_partition_nvariants(partition)

        position = empty(nvariants, dtype=uint32)
        nalleles = empty(nvariants, dtype=uint16)
        var_offset = empty(nvariants, dtype=uint64)
        vid_max_len = ffi.new("uint32_t[]", 1)
        rsid_max_len = ffi.new("uint32_t[]", 1)
        chrom_max_len = ffi.new("uint32_t[]", 1)
        allele_ids_max_len = ffi.new("uint32_t[]", 1)

        position_ptr = ffi.cast("uint32_t *", ffi.from_buffer(position))
        nalleles_ptr = ffi.cast("uint16_t *", ffi.from_buffer(nalleles))
        offset_ptr = ffi.cast("uint64_t *", ffi.from_buffer(var_offset))
        lib.read_partition_par